# Demo: How to Modify Models Using GRADIEND

1. Select a model to modify

In [ ]:
from transformers import AutoModel, AutoTokenizer, pipeline, logging

# Suppress warnings from Hugging Face transformers library
logging.set_verbosity_error()

In [ ]:
from gradiend.combined_models.test_combined_ae import CombinedEncoderDecoder
from gradiend.model import ModelWithGradiend
from hydra import initialize, compose
from gradiend.training.trainer import train

with initialize(config_path="conf", version_base=None):  # adjust path relative to notebook
        # Step 2: Compose the configuration with overrides
        cfg = compose(
            config_name="config",
            overrides=[
                "mode.name=gradiend",
                "pairing=MFN_UL",
                "num_runs=1"
            ]
        )

        MF = "results/experiments/gradiend/MF/1e-05/bert-base-german-cased/0"
        MN = "results/experiments/gradiend/MN/1e-05/bert-base-german-cased/0"
        FN = "results/experiments/gradiend/FN/1e-5/bert-base-german-cased/0"

        MF_model = ModelWithGradiend.from_pretrained(MF)
        base_model = MF_model.base_model
        tokenizer = MF_model.tokenizer
        MN_model = ModelWithGradiend.from_pretrained(MN)
        FN_model = ModelWithGradiend.from_pretrained(FN)

        combined_ae = CombinedEncoderDecoder([MF_model, MN_model, FN_model])
        model_with_gradiend = ModelWithGradiend(base_model, combined_ae, tokenizer)
        model_config = cfg.mode.model_config 
        config = cfg.pairing

        train(model_with_gradiend, config=config, **model_config)



In [ ]:
import yaml

model = 'bert-base-cased'
distilbert_de = 'distilbert-base-german-cased'
bert_de = 'bert-base-german-cased'


2. Train the GRADIEND model

In [ ]:
from gradiend.training.gradiend_training import train

# you may override some default behavior of gradiend.training.trainer.train() with the model_config
model_config = {
    'eval_max_size': 0.1, # use all of the validation data
    'epochs': 1,
    'lr': 1e-4,
}

gradiend_model_dir = train(bert_de,config, model_config, multi_task=False, n=1, dim=2)

3. [Optional]: Analyze the Encoder

In [ ]:
import yaml
from gradiend.evaluation.analyze_encoder import analyze_models
from gradiend.export.encoder_stats import print_encoder_stats
from gradiend.export.encoder_plot import plot

config = yaml.safe_load(open("config.yml"))['M_F_N_leipzig']
gradiend_model_dir = "results/models/gradiend"



In [ ]:
print(config)

In [ ]:

analyze_models(gradiend_model_dir, config=config, multi_task=False, shared=True)

In [ ]:
print_encoder_stats(gradiend_model_dir, config=config)

# plot the encoded values distribution across different datasets

In [ ]:
plot(config, gradiend_model_dir, multi_grad=True)

In [ ]:
gradiend_model_dir = "gradiend/results/experiments/gradiend/multi_task/gradient/latent_2/1e-05/batch_16_nom_only/epoch_3/bert-base-german-cased/0_epoch_3"
config = yaml.safe_load(open("config.yml"))['DAS_PRON']

plot(config, gradiend_model_dir)

4. Analyze the Decoder

In [ ]:
from gradiend.evaluation.analyze_decoder import default_evaluation
default_evaluation(gradiend_model_dir)

5. Create modified models based on the base models by selecting parameters based on the analysis and the BPI, FPI, and MPI metrics

In [ ]:
from gradiend.evaluation.select_models import select
result = select(gradiend_model_dir, force=False, plot=False)

6. Load the modified models and do something with them

In [ ]:
for suffix in ['N', 'F', 'M']:
    model_name = f'results/changed_models/{model}-{suffix}'
    print(f'Loading model {model_name}')
    modified_model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # do something with the model
    # ...
    
    # Example: Use the pipeline to predict the masked word    
    fill_mask = pipeline("fill-mask", model=model_name, tokenizer=model_name)
    text = 'The man worked as a [MASK].'
    result = fill_mask(text)
    predicted = result[0]['token_str']
    predicted_prob = result[0]['score']
    print(f'Predicted for {suffix}: {predicted} ({predicted_prob})')

7. [Optional]: Evaluate the modified models on a simple masking task to evaluate overfitting

In [ ]:
from gradiend.evaluation.analyze_decoder import evaluate_gender_prediction_for_models
from gradiend.export.gender_predictions import plot_all

for targets in [('man', 'woman'), ('woman', 'man')]:
    evaluate_gender_prediction_for_models(model, target_words=targets)
    suffix = '_'.join(targets)
    plot_all(f'results/gender_prediction/{model}.csv', suffix=suffix)

8. [Optional]: Generate some example predictions for the model

In [ ]:
from gradiend.export.example_predictions import run_for_model
run_for_model(model)